## Validating Scalar-Wiedemann Against Brute-Force Nullspace

We’ll generate all vectors over GF(2) for small matrices (n≤4), compute their nullspace by brute force, and compare to the kernel vector returned by `ScalarWiedemann.solve`. This will reveal whether the implementation truly finds a nontrivial solution.

In [5]:
import sys
sys.path.append("./core")
from IPython.utils import io

import numpy as np
from importlib import reload

from gf2 import gf2matrix
from wiedemann import wiedemann
# reload(gf2matrix)
# reload(wiedemann)

In [6]:
import itertools

# Brute-force nullspace over GF(2)

def brute_nullspace(M):
    n = M.n_rows
    nulls = []
    for vec in itertools.product([0,1], repeat=n):
        v = np.array(vec, dtype=np.int8)
        if not np.any(M.apply(v)) and np.any(v):
            nulls.append(v)
    return nulls

# Test small matrices
for n in range(2,5):
    print(f"Testing n={n}")
    # random sparse matrices
    for _ in range(5):
        A = gf2matrix.random(n, density=0.5)
        brute = brute_nullspace(A)
        if not brute:
            continue
        w = wiedemann.solve(A, max_iter=10, verbose=True)
        # Compare
        ok = w is not None and any(np.array_equal(w, v) for v in brute)
        print(f"  Matrix A:")
        print(A)
        print(f"  Brute nullspace has {len(brute)} nontrivial vectors")
        if w is None:
            print("  Wiedemann returned no solution")
        else:
            print("  Wiedemann solution matches brute force?", ok)
    print()

Testing n=2

Attempt 1 with a different random vector x_base:
[0 1]
[array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8)]
[0, 0, 0, 0]
None
Minimal polynomial: None


TypeError: object of type 'NoneType' has no len()

### Why No Solutions Recorded

- The scalar Wiedemann `solve` method finds a nontrivial nullspace vector **only** if the matrix has a nonzero kernel.  
- Random sparse matrices over GF(2) are often full rank (no nonzero nullspace), so `solve` correctly returns `None`.  
- This does _not_ indicate a bug: there simply isn’t a nonzero solution to find.  

Let’s test on a small, explicitly singular matrix where we know a nullspace exists.

In [10]:
# Known singular matrix example
def example_singular():
    print("\nSingular matrix example:")
    A = gf2matrix.from_dense(np.array([[1,1,0],[1,1,0],[0,0,1]], dtype=np.int8))
    print(A)
    print("Expected nullspace: any vector of form [v, v, 0]")
    w = wiedemann.solve(A, max_iter=3, verbose=True)
    if not w:
        print("Wiedemann failed to find a solution")
        return
    print("Computed w:", w)
    print("Verification A·w:", A.apply(w))

example_singular()


Singular matrix example:
[[1 1 0]
 [1 1 0]
 [0 0 1]]
Expected nullspace: any vector of form [v, v, 0]

Attempt 1 with a different random vector x_base:
[1 0 0]
[array([1, 1, 0], dtype=int8), array([0, 0, 0], dtype=int8), array([0, 0, 0], dtype=int8), array([0, 0, 0], dtype=int8), array([0, 0, 0], dtype=int8), array([0, 0, 0], dtype=int8)]
[0, 0, 0, 0, 0, 0]
None
Minimal polynomial: None


TypeError: object of type 'NoneType' has no len()

### Why Scalar-Wiedemann May Still Miss Singular Kernels

Even when the matrix is singular, the **scalar** Wiedemann algorithm can fail to find a nullspace vector because it computes the minimal polynomial of the **projected** sequence:

```
S_y[i] = y^T · M^i · u
```

For a singular matrix, detecting the zero eigenvalue requires that `y^T · u` be nonzero on some part of the nullspace. If the random left vector `y` is orthogonal to the actual nullspace component of `u`, then the projected sequence never “sees” the zero eigenvalue and its minimal polynomial will not have `x` as a factor. As a result:
- The scalar minimal polynomial has constant term 1 (no zero-root).
- Kernel reconstruction yields a trivial solution.

**Takeaway:**
- The scalar (1×1) approach is *not guaranteed* to detect a nullspace even if one exists.
- To reliably find nullspace vectors for singular matrices, use the **Block Wiedemann** algorithm (choose multiple left projections) or repeat with many different random `(u,y)` pairs until one “hits” the nullspace.
